## Please read the blog post, and codealong in this Jupyter notebook. https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775                                                                                                   



Please refer to "Recursion" from https://cr.yp.to/2005-261/bender2/DT.pdf , https://medium.com/greyatom/decision-trees-a-simple-way-to-visualize-a-decision-dc506a403aeb, https://www.ke.tu-darmstadt.de/lehre/archiv/ws0809/mldm/dt.pdf

In [2]:
import numpy as np

class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def _best_split(self, X, y):
        """Find the best split for a node.
        "Best" means that the average impurity of the two children, weighted by their
        population, is the smallest possible. Additionally it must be less than the
        impurity of the current node.
        To find the best split, we loop through all the features, and consider all the
        midpoints between adjacent training samples as possible thresholds. We compute
        the Gini impurity of the split generated by that particular feature/threshold
        pair, and return the pair with smallest impurity.
        Returns:
            best_idx: Index of the feature for best split, or None if no split is found.
            best_thr: Threshold to use for the split, or None if no split is found.
        """
        # Need at least two elements to split a node.
        m = y.size
        if m <= 1:
            return None, None

        # Count of each class in the current node.
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]

        # Gini of current node.
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None

        # Loop through all features.
        for idx in range(self.n_features_):
            # Sort data along selected feature.
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))

            # We could actually split the node according to each feature/threshold pair
            # and count the resulting population for each class in the children, but
            # instead we compute them in an iterative fashion, making this for loop
            # linear rather than quadratic.
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):  # possible split positions
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                )

                # The Gini impurity of a split is the weighted average of the Gini
                # impurity of the children.
                gini = (i * gini_left + (m - i) * gini_right) / m

                # The following condition is to make sure we don't try to split two
                # points with identical values for that feature, as it is impossible
                # (both have to end up on the same side of a split).
                if thresholds[i] == thresholds[i - 1]:
                    continue

                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2  # midpoint

        return best_idx, best_thr

::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [3]:
# example data: X = age, y = likes pineapples on pizza (0=no, 1=yes, 2=maybe)
import numpy as np
import pandas as pd


X = pd.DataFrame(data=[10, 10, 20, 30, 40, 50, 60], index=None, columns=['age'])
y = np.array([0, 0, 0, 1, 1, 2, 2])  # 0=no, 1=yes, 2=maybe
n_classes = len(set(y))
n_features = 1
y.size, n_classes

(7, 3)

In [4]:
def best_split(X, y):
    
    # Need at least two elements to split a node.
    m = y.size
    if m <= 1:
        return None, None

    # Count of each class in the current node.
    num_parent = [np.sum(y == c) for c in range(n_classes)]
    print('num_parent: ', num_parent)
    
    # Gini of current node.
    best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
    best_idx, best_thr = None, None
    print('best_gini:  ', best_gini.round(3))

    # Loop through all features.
    for idx in range(n_features):
        # Sort data along selected feature.
        thresholds, classes = zip(*sorted(zip(X.iloc[:, idx], y)))
        print('feature index:  ', idx)

        # We could actually split the node according to each feature/threshold pair
        # and count the resulting population for each class in the children, but
        # instead we compute them in an iterative fashion, making this for loop
        # linear rather than quadratic.
        num_left = [0] * n_classes
        num_right = num_parent.copy()
        
        print('\nbefore any split:')
        print('=================================')
        print('num_left:   ', num_left)
        print('num_right:  ', num_right)
        
        for i in range(1, m):  # possible split positions
            c = classes[i - 1]
            num_left[c] += 1
            num_right[c] -= 1
            gini_left = 1.0 - sum(
                (num_left[x] / i) ** 2 for x in range(n_classes)
            )
            gini_right = 1.0 - sum(
                (num_right[x] / (m - i)) ** 2 for x in range(n_classes)
            )

            print('\nafter possible split at pos. {}:'.format(i))
            print('=================================')
            print('X-value[{}]:  {}'.format(i-1, thresholds[i-1]))
            print('num_left:   ', num_left)
            print('num_right:  ', num_right)
            print('gini_left:   {:.3f}'.format(gini_left))
            print('gini_right:  {:.3f}'.format(gini_right))

            # The Gini impurity of a split is the weighted average of the Gini
            # impurity of the children.
            gini = (i * gini_left + (m - i) * gini_right) / m
            
            print('gini total:  {:.3f}'.format(gini))

            # The following condition is to make sure we don't try to split two
            # points with identical values for that feature, as it is impossible
            # (both have to end up on the same side of a split).
            if thresholds[i] == thresholds[i - 1]:
                print('\n~~~ current threshold ({}) same as next threshold ({}) ~~~'.format(
                    thresholds[i], thresholds[i - 1]))
                print('Hence, continue directly with next position...')
                continue

            if gini < best_gini:
                print('\n\033[1m~~~ gini < best_gini !!! ~~~')
                best_gini = gini
                best_idx = idx
                best_thr = (thresholds[i] + thresholds[i - 1]) / 2  # midpoint
                
                print('new best_gini:  {:.3f}'.format(best_gini))
                print('new best_idx:   {}'.format(best_idx))    
                print('new best_thr:   {} (midpoint between this threshold ({}) and next threshold ({}))\033[0m'.format(
                    best_thr, thresholds[i-1],thresholds[i]))  

    return best_idx, best_thr

In [5]:
# now call the function, which will print a protocol of all important steps that were performed
best_split(X, y)

num_parent:  [3, 2, 2]
best_gini:   0.653
feature index:   0

before any split:
num_left:    [0, 0, 0]
num_right:   [3, 2, 2]

after possible split at pos. 1:
X-value[0]:  10
num_left:    [1, 0, 0]
num_right:   [2, 2, 2]
gini_left:   0.000
gini_right:  0.667
gini total:  0.571

~~~ current threshold (10) same as next threshold (10) ~~~
Hence, continue directly with next position...

after possible split at pos. 2:
X-value[1]:  10
num_left:    [2, 0, 0]
num_right:   [1, 2, 2]
gini_left:   0.000
gini_right:  0.640
gini total:  0.457

~~~ gini < best_gini !!! ~~~
new best_gini:  0.457
new best_idx:   0
new best_thr:   15.0 (midpoint between this threshold (10) and next threshold (20))

after possible split at pos. 3:
X-value[2]:  20
num_left:    [3, 0, 0]
num_right:   [0, 2, 2]
gini_left:   0.000
gini_right:  0.500
gini total:  0.286

~~~ gini < best_gini !!! ~~~
new best_gini:  0.286
new best_idx:   0
new best_thr:   25.0 (midpoint between this threshold (20) and next threshold (30))

af

(0, 25.0)

### https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775